# Movie Recommendor System 

**What recommendor system does?**

It is used to recommend similar products that are searched by customer.

**Types of recommendor system**

1. **Content Based** - It recommend product that are of same(similar) content. In this we create tags and on the basis of those      tags we recommend products.
     example - If you are listning romantic song then it will recommend other romantic song.
   
2. **Collaborative filtering based** - It recommend product on the basis of user's interest , let say there are two friends and      they are very similar like example they saw a movie and they have same opinion about the same movie ,let say one friend saw      movie m2 and he likes that movie we know that we know that second friend have same similarity then we will recommend m2 to      second friend.

3. **Hybrid based** - It is a mixture of Content Bases and Collaborative filtering based.

In [376]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [377]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [378]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [379]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [380]:
movies=movies.merge(credits,on='title')

In [381]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [382]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

# Important columns for recommendor system 
**1.Genres
  2.id
  3.keywords
  4.overview
  5.release_date
  6.title
  7.cast
  8.crew**

In [383]:
movies=movies[['genres','movie_id','cast','crew','release_date','title','overview','keywords']]

In [384]:
movies.isnull().sum()

genres          0
movie_id        0
cast            0
crew            0
release_date    1
title           0
overview        3
keywords        0
dtype: int64

In [385]:
movies.dropna(inplace=True)

In [386]:
duplicate_df_rows = movies[movies.duplicated()]
print(f" no. of duplicate rows : {duplicate_df_rows.shape}")

 no. of duplicate rows : (0, 8)


In [387]:
movies.head(1)

,genres,movie_id,cast,crew,release_date,title,overview,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009-12-10,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."


In [388]:
movies.iloc[0].genres  # we need to convert this dictionery into list like ["Action","Adventure", "Fantasy","Science Fiction"]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [390]:
import ast     # this function solve the problem of "string indices must be integers"
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [391]:
def helper(obj):  # TO EXTRACT THE NAME OF GENRE ONLY 
    list=[]
    for i in ast.literal_eval(obj):
        list.append(i["name"])
    return list

In [392]:
movies['genres']=movies['genres'].apply(helper)

In [393]:
movies['keywords']=movies['keywords'].apply(helper)

In [394]:
def helper_1(obj):  # to extract the first three name in cast
    list=[]
    j=0
    for i in ast.literal_eval(obj):
        if j!=3:
            list.append(i["name"])
            j=j+1
        else:
            break    
    return list

In [395]:
movies['cast']=movies['cast'].apply(helper_1)

In [396]:
movies.head(2)

,genres,movie_id,cast,crew,release_date,title,overview,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009-12-10,Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon..."
1,"[Adventure, Fantasy, Action]",285,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",2007-05-19,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ..."


In [397]:
def helper_2(obj):  # To extract only director name in crew 
    list=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            list.append(i["name"])
            break   
    return list

In [398]:
movies['crew']=movies['crew'].apply(helper_2) 

In [399]:
movies.head(2)

,genres,movie_id,cast,crew,release_date,title,overview,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],2009-12-10,Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon..."
1,"[Adventure, Fantasy, Action]",285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],2007-05-19,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ..."


In [400]:
movies['overview']=movies['overview'].apply(lambda x:x.split()) # to convert str into list

In [339]:
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [401]:
movies.head()

,genres,movie_id,cast,crew,release_date,title,overview,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],2009-12-10,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[culture clash, future, space war, space colon..."
1,"[Adventure, Fantasy, Action]",285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],2007-05-19,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drug abuse, exotic island, east india ..."
2,"[Action, Adventure, Crime]",206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],2015-10-26,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[spy, based on novel, secret agent, sequel, mi..."
3,"[Action, Crime, Drama, Thriller]",49026,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],2012-07-16,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[dc comics, crime fighter, terrorist, secret i..."
4,"[Action, Adventure, Science Fiction]",49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],2012-03-07,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[based on novel, mars, medallion, space travel..."


In [402]:
movies['tags']=movies['overview']+movies['keywords']+ movies['crew']+ movies['genres']+ movies['cast']

In [403]:
movies.head()

,genres,movie_id,cast,crew,release_date,title,overview,keywords,tags
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],2009-12-10,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin..."
1,"[Adventure, Fantasy, Action]",285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],2007-05-19,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drug abuse, exotic island, east india ...","[Captain, Barbossa,, long, believed, to, be, d..."
2,"[Action, Adventure, Crime]",206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],2015-10-26,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[spy, based on novel, secret agent, sequel, mi...","[A, cryptic, message, from, Bond’s, past, send..."
3,"[Action, Crime, Drama, Thriller]",49026,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],2012-07-16,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[dc comics, crime fighter, terrorist, secret i...","[Following, the, death, of, District, Attorney..."
4,"[Action, Adventure, Science Fiction]",49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],2012-03-07,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[based on novel, mars, medallion, space travel...","[John, Carter, is, a, war-weary,, former, mili..."


In [404]:
new_df = movies[['movie_id','title','tags']]

In [405]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [406]:
new_df['tags']=new_df['tags'].apply(lambda x:" " .join(x))     # to convert list into string 

C:\Users\Asus\AppData\Local\Temp/ipykernel_13412/2121210845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" " .join(x))     # to convert list into string


In [407]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [408]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())  # convert every letter into small letter 

C:\Users\Asus\AppData\Local\Temp/ipykernel_13412/1923550984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())  # convert every letter into small letter


In [409]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


# we are done with preprocessing , now we will do the text vectorization 

# Text vectorization 
Use to convert text into vector form 

In [410]:
!pip install nltk

In [411]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [412]:
def stem(text):  # for stamming the data 
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [413]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\Asus\AppData\Local\Temp/ipykernel_13412/3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [414]:
from sklearn.feature_extraction.text import CountVectorizer   # to choose 5000 common words 
cv=CountVectorizer(max_features=5000,stop_words = 'english')

In [415]:
vectors=cv.fit_transform(new_df['tags']).toarray() # toarray is used to convert scipy sparse matrix into numpy array

In [416]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [417]:
cv.get_feature_names() # this are the top 5000 words

['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '16th',
 '17',
 '17th',
 '18',
 '18th',
 '19',
 '1910',
 '1920',
 '1930',
 '1940',
 '1950',
 '1950s',
 '1960',
 '1960s',
 '1970',
 '1970s',
 '1980',
 '1985',
 '1990',
 '19th',
 '20',
 '200',
 '2009',
 '20th',
 '21st',
 '24',
 '25',
 '30',
 '300',
 '3d',
 '40',
 '50',
 '500',
 '51',
 '60',
 '70',
 'aaron',
 'abandon',
 'abbi',
 'abduct',
 'abigail',
 'abil',
 'abl',
 'aboard',
 'aborigin',
 'abov',
 'abraham',
 'abroad',
 'abus',
 'academi',
 'accent',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 'accompani',
 'accomplish',
 'account',
 'accus',
 'ace',
 'achiev',
 'act',
 'action',
 'activ',
 'activist',
 'activities',
 'actor',
 'actress',
 'actual',
 'ad',
 'adam',
 'adapt',
 'add',
 'addict',
 'adjust',
 'admir',
 'admit',
 'adolesc',
 'adolf',
 'adopt',
 'ador',
 'adrian',
 'adrien',
 'adult',
 'adulteri',
 'adulthood',
 'advanc',
 'adventur',
 'adventure',
 'advertis',
 'advic',
 'advis',
 'affa

Now we need to calculate the distance between each vector , the more the distance the less the similarity

In [418]:
from sklearn.metrics.pairwise import cosine_similarity # to calculate the cosine distance

In [419]:
similarity=cosine_similarity(vectors)  

**Now we will make a function that takes movie as a input and give 5 similar movies as an output**

In [420]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [421]:
recommend('Avatar')

Aliens
Silent Running
Moonraker
Alien
Mission to Mars


In [422]:
import pickle

In [423]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [424]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [425]:
pickle.dump(similarity,open('similarity.pkl','wb'))